In [1]:
import pandas as pd
import lightgbm as lgb
import os
import gc
import pickle
from sklearn import preprocessing, model_selection

In [2]:
#"""
def get_type_feature_all_(sample, train_df, key, on, type_c, mark):
    if type_c == "count":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].count()).reset_index()
    if type_c == "mean":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].mean()).reset_index()
    if type_c == "nunique":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].nunique()).reset_index()
    if type_c == "max":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].max()).reset_index()
    if type_c == "min":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].min()).reset_index()
    if type_c == "sum":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].sum()).reset_index()
    if type_c == "std":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].std()).reset_index()
    if type_c == "median":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].median()).reset_index()
    tmp.columns = key + [mark + "_" + "_".join(key) + '_%s_' % type_c + on]
    tmp[mark + "_" + "_".join(key) + '_%s_' % type_c + on] = tmp[
        mark + "_" + "_".join(key) + '_%s_' % type_c + on].astype('float32')
    sample = sample.merge(tmp, on=key, how='left')
    del tmp, train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on
#"""
def get_type_feature_all(sample,train_df,key,on,type_c,mark):
    pickle_path="../pickle/"
    filename = "_".join([mark+"_%s_features"%type_c, "_".join(key), on, str(len(sample))]) + ".pkl"
    try:
        with open(pickle_path + filename, "rb") as fp:
            print("load {} {} feature from pickle file: key: {}, on: {}...".format(mark,type_c,"_".join(key), on))
            col = pickle.load(fp)
        for c in col.columns:
            sample[c] = col[c]
        gc.collect()
    except:
        print('get {} {} feature, key: {}, on: {}'.format(mark,type_c,"_".join(key), on))
        if type_c=="count":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].count()).reset_index()
        if type_c=="mean":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].mean()).reset_index()
        if type_c=="nunique":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].nunique()).reset_index()
        if type_c=="max":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].max()).reset_index()
        if type_c=="min":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].min()).reset_index()
        if type_c=="sum":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].sum()).reset_index()
        if type_c=="std":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].std()).reset_index()
        if type_c=="median":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].median()).reset_index()
        tmp.columns = key+[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
        tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on] = tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on].astype('float32')
        sample = sample.merge(tmp, on=key, how='left')
        with open(pickle_path + filename, "wb") as fp:
            col = sample[[mark+"_"+"_".join(key) + '_%s_'%type_c + on]].copy()
            pickle.dump(col, fp)
        del tmp
    #if save_only:
        #for i in col.columns:
            #del sample[i]
    del col,train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on


In [3]:
train_data=pd.read_csv("../data/train.csv")
test_data=pd.read_csv("../data/test.csv")
sub=pd.read_csv("../data/sample_submission.csv")

In [4]:
train_data.columns

Index(['article_id', 'date', 'baike_id_1h', 'price', 'price_diff', 'author',
       'level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url',
       'comments_1h', 'zhi_1h', 'buzhi_1h', 'favorite_1h', 'orders_1h',
       'baike_id_2h', 'comments_2h', 'zhi_2h', 'buzhi_2h', 'favorite_2h',
       'orders_2h', 'orders_3h_15h'],
      dtype='object')

In [5]:
def create_features(data):
    data["order_1h_2h"]=data["orders_1h"]+data["orders_2h"]
    
    date_max=data.groupby(["date"]).article_id.max().reset_index()
    date_max.columns=["date","date_max"]
    data_min=date_max.copy()
    data_min.columns=["date","date_min"]
    data_min["date"]=data_min["date"]+1
    
    data=data.merge(date_max,on="date",how="left")
    data=data.merge(data_min,on="date",how="left")
    data["date_min"]=data["date_min"].fillna(0)
    
    data["article_id"]=(data["article_id"]-data["date_min"])/(data["date_max"]-data["date_min"])
    
    data['hour']=data["article_id"].apply(lambda x:int(x*24))
    
    data["dayofweek"]=data["date"]%7
    
    del data["date_min"],data["date_max"]
    
    
    cat_features=[['baike_id_1h'],['author'],['level1'], ['level2'], ['level3'], ['level4'], ['brand'], ['mall'], ['url'],['dayofweek'],
                  ['author','level1'],['author','level2'],['author','level3'],['author','level4'],['author','brand'],['author','mall'],
                  ['mall','brand'],['mall','level1'],['mall','level2'],['mall','level3'],['mall','level4'],
                  ['brand','level1'],['brand','level2'],['brand','level3'],['brand','level4'],
                  ['baike_id_1h','mall'],
                  ["mall","brand","level1"],["mall","brand","level2"],["mall","brand","level3"],["mall","brand","level4"]

                 ]
    num_features=['comments_1h','comments_2h',
                  'zhi_1h','zhi_2h',
                  'buzhi_1h','buzhi_2h',
                  'favorite_1h','favorite_2h',
                  'orders_1h','orders_2h','order_1h_2h','article_id']
    
    for cat in cat_features:
        data["%s_%s_-1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-1)-data["article_id"]
        data["%s_%s_1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(1)-data["article_id"]

        data,_=get_type_feature_all(data, data, cat,"%s_%s_-1"%("_".join(cat),"article_id") , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"%s_%s_1"%("_".join(cat),"article_id") , "mean", "fe")
    
    for cat in cat_features:
        data,_=get_type_feature_all(data, data, cat+["hour"],"article_id" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"article_id" , "count", "fe")
        
        data,_=get_type_feature_all(data, data, cat,"price" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "max", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "min", "fe")
        

        for num in num_features:
            data,_=get_type_feature_all(data, data, cat,num , "mean", "fe")
            data,_=get_type_feature_all(data, data, cat,num , "sum", "fe")

    
    for cat in cat_features:
        data["%s_%s_-1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-1)-data["article_id"]
        data["%s_%s_1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(1)-data["article_id"]

        data["%s_%s_-2"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-2)-data["article_id"]
        data["%s_%s_2"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(2)-data["article_id"]
        
        data["%s_%s_-3"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-3)-data["article_id"]
        data["%s_%s_-4"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-4)-data["article_id"]
        data["%s_%s_-5"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-5)-data["article_id"]

        data["%s_%s_3"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(3)-data["article_id"]
        data["%s_%s_4"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(4)-data["article_id"]
        #data,_=get_type_feature_all(data, data, cat+["orders_2h"],"article_id" , "count", "fe")
        #"""
        for num in ['orders_1h','orders_2h','order_1h_2h']:
            data["%s_%s_-1"%("_".join(cat),num)]=data.groupby(cat)[num].shift(-1)
            data["%s_%s_1"%("_".join(cat),num)]=data.groupby(cat)[num].shift(1)
        #data["%s_%s_-1"%("_".join(cat),'price')]=data.groupby(cat)['price'].shift(-1)-data['price']
            #data["%s_%s_date_-1"%("_".join(cat),num)]=data.groupby(cat+["date"])[num].shift(-1)
    
    return data

In [6]:
data=pd.concat([train_data,
                #test_data,
               ]).reset_index(drop=True)

data["orders_1h_2h"]=data["orders_1h"]+data["orders_2h"]
#"""
features_data=data[data.date<110].copy()
def get_data(dt):
    data_tmp=data[data.date==dt].copy()
    features=features_data[features_data.date!=dt].copy()
    features["r1"]=features["orders_1h"]/features["orders_3h_15h"]
    features["r2"]=features["orders_2h"]/features["orders_3h_15h"]
    features["r3"]=features["orders_1h_2h"]/features["orders_3h_15h"]
    for cat in ['baike_id_1h','author','level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url']:
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"orders_3h_15h" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r1" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r2" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r3" , "mean", "features")
    return data_tmp



data_all=[]
for dt in range(1,124):
    data_tmp=get_data(dt)
    data_all.append(data_tmp)
data_all=pd.concat(data_all).reset_index(drop=True)
#"""
data=create_features(data_all)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe mean feature from pickle file: key: level4, on: buzhi_2h...
load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pic

load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: author_level1, on: comments_1h...
load fe sum feature from pickle file: key:

load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe mean feature from pickle file: key: author_level4, on: article_id...
load fe sum feature from pickle file: key: author_level4, on: article_id...
load f

load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_level1, on: orders_1h...
load fe mean feature from pickle file: key: mall_level1, on: orders_2h...
load fe sum feature from pickle file: key: mall_level1, on: orders_2h...
load fe mean feature from pickle file: key: 

load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_1h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_2h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_2h...
load fe mean feature from pickle file:

load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: buzhi_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: buzh

load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle file: key: mall_brand_level3, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4_hour, on: article_id...
load fe count feature from pickle file: key: mall_brand_level4, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4, on: price...
load fe max feature from pickle file: key: mall_brand_level4, on: price...
load fe min feature from pickle file: k

In [7]:
print(data.shape)

(1807657, 1445)


In [8]:
train=data[(data.date<110)].copy()
#train=data[data.date<117].copy()
#train=data[(data.date>=103)&(data.date<110)].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()

In [9]:
features = list(train[:1].drop(['date', 'orders_3h_15h'],axis=1).columns)

In [10]:
train_set = lgb.Dataset(train[features], label=train["orders_3h_15h"])
val_set = lgb.Dataset(valid[features], label=valid["orders_3h_15h"])

params_lgbm = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'max_depth': 5,
          'lambda_l2': 0.7,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.03,
          #'tree_method': 'exact',
          'seed': 2021,
          'nthread': 18,
          'verbose': -1,
          }


model = lgb.train(params_lgbm, 
                  train_set, 
                  5000, 
                  valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  verbose_eval= 500,
                  #early_stopping_rounds=500,
                  #categorical_feature=cats,
                 )

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[500]	valid_0's l2: 1.09465
[1000]	valid_0's l2: 1.08075
[1500]	valid_0's l2: 1.07507
[2000]	valid_0's l2: 1.07196
[2500]	valid_0's l2: 1.07005
[3000]	valid_0's l2: 1.06877
[3500]	valid_0's l2: 1.06758
[4000]	valid_0's l2: 1.06677
[4500]	valid_0's l2: 1.06627
[5000]	valid_0's l2: 1.06596


In [11]:
valid_score=train_data[(train_data.date>=110)].copy()[['article_id', 'date','orders_1h','orders_2h','orders_3h_15h']].copy()
valid_score["pred"]=model.predict(valid[features])
valid_score["pred"]=valid_score["pred"].apply(lambda x:0 if x<0 else x)
score=((valid_score["pred"]-valid_score["orders_3h_15h"])**2).mean()
print(score)
valid_score[["article_id","pred"]].to_csv("../user_data/train_stacking_%s.csv"%str(score),index=None)

1.0638693813136202


In [12]:
#[5000]	valid_0's l2: 1.06644
#[5000]	valid_0's l2: 1.0692
#[1000]	valid_0's l2: 1.08886
#[1000]	valid_0's l2: 1.1011            +sum
#[1000]	valid_0's l2: 1.1043            +zhi,buzhi,fav,com
#[1000]	valid_0's l2: 1.10834
#[1000]	valid_0's l2: 1.10897           +price
#[1000]	valid_0's l2: 1.11109    alld   +weekday
#[1000]	valid_0's l2: 1.11466    30d
#[1000]	valid_0's l2: 1.11559    alld

In [13]:
print("\n".join(("%s: %.2f" % x) for x in
                list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1],
                      
                            reverse=True))[:200]
                ))


orders_1h_2h: 12524062.65
orders_2h: 8444863.99
order_1h_2h: 6222569.03
fe_url_mean_order_1h_2h: 3119915.54
features_url_mean_orders_3h_15h: 3105008.48
favorite_2h: 1291421.82
features_author_mean_orders_3h_15h: 878438.54
features_mall_mean_orders_3h_15h: 783149.86
fe_dayofweek_hour_mean_article_id: 691598.09
features_baike_id_1h_mean_orders_3h_15h: 464814.48
dayofweek_article_id_-5: 426527.26
article_id: 389005.43
fe_baike_id_1h_mall_hour_mean_article_id: 349482.97
fe_baike_id_1h_hour_mean_article_id: 335624.85
dayofweek_article_id_-4: 300384.40
fe_author_brand_mean_order_1h_2h: 289952.62
fe_mall_mean_price: 260636.75
fe_author_brand_mean_orders_2h: 240902.13
comments_2h: 221680.02
zhi_2h: 216525.21
price_diff: 212961.36
favorite_1h: 211589.15
fe_mall_hour_mean_article_id: 194565.90
fe_url_sum_order_1h_2h: 187347.55
fe_mall_brand_level3_mean_orders_2h: 178173.37
orders_1h: 166470.19
fe_author_brand_hour_mean_article_id: 166072.23
fe_mall_level4_hour_mean_article_id: 158908.54
fe_autho

In [14]:
data=pd.concat([train_data,
                test_data,
               ]).reset_index(drop=True)

data["orders_1h_2h"]=data["orders_1h"]+data["orders_2h"]
#"""
features_data=data[data.date<117].copy()
def get_data(dt):
    data_tmp=data[data.date==dt].copy()
    features=features_data[features_data.date!=dt].copy()
    features["r1"]=features["orders_1h"]/features["orders_3h_15h"]
    features["r2"]=features["orders_2h"]/features["orders_3h_15h"]
    features["r3"]=features["orders_1h_2h"]/features["orders_3h_15h"]
    for cat in ['baike_id_1h','author','level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url']:
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"orders_3h_15h" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r1" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r2" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r3" , "mean", "features")
    return data_tmp



data_all=[]
for dt in range(1,124):
    data_tmp=get_data(dt)
    data_all.append(data_tmp)
data_all=pd.concat(data_all).reset_index(drop=True)
#"""
data=create_features(data_all)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe mean feature from pickle file: key: level4, on: buzhi_2h...
load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pic

load fe mean feature from pickle file: key: dayofweek, on: orders_1h...
load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: auth

load fe sum feature from pickle file: key: author_level4, on: buzhi_1h...
load fe mean feature from pickle file: key: author_level4, on: buzhi_2h...
load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe me

load fe mean feature from pickle file: key: mall_level1, on: zhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: zhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: zhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_leve

load fe mean feature from pickle file: key: brand_level1, on: comments_1h...
load fe sum feature from pickle file: key: brand_level1, on: comments_1h...
load fe mean feature from pickle file: key: brand_level1, on: comments_2h...
load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file:

load fe sum feature from pickle file: key: brand_level4, on: article_id...
load fe mean feature from pickle file: key: baike_id_1h_mall_hour, on: article_id...
load fe count feature from pickle file: key: baike_id_1h_mall, on: article_id...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: price...
load fe max feature from pickle file: key: baike_id_1h_mall, on: price...
load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, o

load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle file: key: mall_brand_level3, on: article_id...
load fe mean feature from pickle

In [15]:
train=data[data.date<117].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()
test=data[data.date>=117].copy()

In [16]:
train_set = lgb.Dataset(train[features], label=train["orders_3h_15h"])
val_set = lgb.Dataset(valid[features], label=valid["orders_3h_15h"])

model = lgb.train(params_lgbm, 
                  train_set, 
                  5000, 
                  valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  verbose_eval= 500,
                  #early_stopping_rounds=500,
                 )

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[500]	valid_0's l2: 1.05548
[1000]	valid_0's l2: 1.02821
[1500]	valid_0's l2: 1.01024
[2000]	valid_0's l2: 0.99528
[2500]	valid_0's l2: 0.982615
[3000]	valid_0's l2: 0.970341
[3500]	valid_0's l2: 0.959182
[4000]	valid_0's l2: 0.94831
[4500]	valid_0's l2: 0.938176
[5000]	valid_0's l2: 0.928662


In [17]:
submission=test_data[["article_id"]].copy()
submission["orders_3h_15h"]=model.predict(test[features])
submission["orders_3h_15h"]=submission["orders_3h_15h"].apply(lambda x:0 if x<0 else x)
#submission["orders_3h_15h"]=submission["orders_3h_15h"]+1000000
submission.to_csv("../sub/summission_%s.csv"%score,index=None)
submission.to_csv("../sub/summission.csv",index=None)
submission["pred"]=submission["orders_3h_15h"]
submission[["article_id","pred"]].to_csv("../user_data/test_stacking_%s.csv"%str(score),index=None)